La red neuronal que armamos no cuenta con capas intermedias o ocultas. Solo tiene una capa de entrada, basada en un dataset descargado de Kaggle que contiene. Esta contiene atributos como tamaño, peso, dulzura, crujiente y jugosidad. La salida de la red intenta predecir tres clases no excluyentes para cada fruta: si es "dulce", "jugosa" y/o "crujiente"

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [10]:
#Utilizamos una función de activación sigmoide en la salida, para la clasificación binaria de cada clase.
#La función sigmoide se usa como función de activación en la capa de salida, generando valores entre 0 y 1.
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

#Fuencion derivada de la sigmoide
def sigmoid_derivative(x):
    return x * (1 - x)

#Funciones de calculo y error
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [11]:
#Inicialización de pesos y bias de forma aleatoria
def initialize_weights(input_size, output_size):
    weights = np.random.randn(input_size, output_size) * np.sqrt(2. / input_size)
    bias = np.zeros((1, output_size))
    return weights, bias


In [12]:
#Forward Pass
#Aca calculo la salida de la red aplicando la función sigmoide después de multiplicar las entradas por los pesos y sumar el bias.
def forward_pass(X, weights, bias):
    z = np.dot(X, weights) + bias
    return sigmoid(z)

In [13]:
#Backpropagation
#Ajusta los pesos y bias basándose en el error, utilizando la derivada de la función de activación.
def backpropagation(X, y, output, weights, bias, learning_rate):
    error = output - y
    gradient = error * sigmoid_derivative(output)
    d_weights = np.dot(X.T, gradient) / X.shape[0]
    d_bias = np.sum(gradient, axis=0, keepdims=True) / X.shape[0]
    weights -= learning_rate * d_weights
    bias -= learning_rate * d_bias
    return weights, bias

In [14]:
#Entrenamiento
#Esto lo que hace es itera en varias épocas (ciclos de entrenamiento),
#ajustando los pesos y bias en cada ciclo para minimizar el error.
def train(X, y, input_size, output_size, epochs, learning_rate):
    weights, bias = initialize_weights(input_size, output_size)
    for epoch in range(epochs):
        output = forward_pass(X, weights, bias)
        loss = mean_squared_error(y, output)
        weights, bias = backpropagation(X, y, output, weights, bias, learning_rate)
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")
    return weights, bias

In [15]:
#Predicción
#Al finalizar el entrenamiento, genera predicciones usando la función de activación en la capa de salida.
#Las predicciones se redondean para obtener valores binarios (0 o 1).
def predict(X, weights, bias):
    output = forward_pass(X, weights, bias)
    return (output > 0.5).astype(int)

In [16]:
#Importo el dataset
#Escogemos solo las columnas Size, Weight, Sweetness, Crunchiness, y Juiciness
#que son las características de cada fruta en este caso.
#Las columnas Sweetness, Juiciness y Crunchiness se convierten en etiquetas binarias (1 o 0)
fruit = pd.read_csv('apple_quality.csv')
fruit['Acidity'] = pd.to_numeric(fruit['Acidity'], errors='coerce')
fruit = fruit.dropna()
fruit['Sweet'] = np.where(fruit['Sweetness'] > 0.5, 1, 0).astype(float)
fruit['Juicy'] = np.where(fruit['Juiciness'] > 0.5, 1, 0).astype(float)
fruit['Crunchy'] = np.where(fruit['Crunchiness'] > 0.5, 1, 0).astype(float)
features = ['Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness']
scaler = StandardScaler()
X = scaler.fit_transform(fruit[features])
Y = fruit[['Sweet', 'Juicy', 'Crunchy']].values

In [17]:
#En esta lista guardamos los nombres de las columnas que serán usadas como entrada para la red.
#transforma los valores de las características para que tengan 0 y 1.
#X contiene las que serán las entradas de la red
#Y contiene las etiquetas binarias que van a ser la salida.
features = ['Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness']
scaler = StandardScaler()
X = scaler.fit_transform(fruit[features])
Y = fruit[['Sweet', 'Juicy', 'Crunchy']].values

In [18]:
#Ponemos los Hiperparametros
input_size = X.shape[1]
output_size = Y.shape[1]
epochs = 3000
learning_rate = 0.001

In [19]:
#Aca hacemos entrenar la red neuronal
#Inicializa los pesos y el sesgo.
#Realiza la propagación hacia adelante y hacia atrás en cada para ajustar los pesos y el sesgo.
weights, bias = train(X, Y, input_size, output_size, epochs, learning_rate)

Epoch 0, Loss: 0.1991
Epoch 100, Loss: 0.1977
Epoch 200, Loss: 0.1964
Epoch 300, Loss: 0.1951
Epoch 400, Loss: 0.1938
Epoch 500, Loss: 0.1925
Epoch 600, Loss: 0.1912
Epoch 700, Loss: 0.1900
Epoch 800, Loss: 0.1887
Epoch 900, Loss: 0.1875
Epoch 1000, Loss: 0.1863
Epoch 1100, Loss: 0.1851
Epoch 1200, Loss: 0.1839
Epoch 1300, Loss: 0.1828
Epoch 1400, Loss: 0.1816
Epoch 1500, Loss: 0.1805
Epoch 1600, Loss: 0.1794
Epoch 1700, Loss: 0.1783
Epoch 1800, Loss: 0.1772
Epoch 1900, Loss: 0.1761
Epoch 2000, Loss: 0.1751
Epoch 2100, Loss: 0.1740
Epoch 2200, Loss: 0.1730
Epoch 2300, Loss: 0.1720
Epoch 2400, Loss: 0.1710
Epoch 2500, Loss: 0.1700
Epoch 2600, Loss: 0.1690
Epoch 2700, Loss: 0.1681
Epoch 2800, Loss: 0.1671
Epoch 2900, Loss: 0.1662


In [20]:
#Predicciones
#uso el modelo entrenado (con los pesos y sesgo ajustados) para hacer predicciones en X
predictions = predict(X, weights, bias)
print("Predicciones:")
print(predictions)

#Mostramos las Y para ver  las predicciones y evaluar la funcionalidad de la red.
print("Salidas reales:")
print(Y)

#calculamos el error cuadrático medio entre las salidas reales y las predicciones.
#Nos da una idea de cuánto se desvían las predicciones de la red neuronal respecto a las etiquetas reales.
mse = mean_squared_error(Y, predictions)
print("Mean Squared Error:", mse)

Predicciones:
[[1 1 0]
 [1 1 1]
 [0 1 0]
 ...
 [0 0 0]
 [1 1 0]
 [1 1 0]]
Salidas reales:
[[1. 1. 0.]
 [1. 1. 1.]
 [0. 1. 0.]
 ...
 [0. 1. 1.]
 [1. 1. 0.]
 [0. 1. 0.]]
Mean Squared Error: 0.19625


Nosotras hicimios una red neuronal para clasificar frutas en función de sus características, utilizando un conjunto de datos que incluía atributos como tamaño, peso, dulzura, crujiente y jugosidad. Despues de preparar los datos y normalizar las caracteristicas y poner las etiquetas pudimos hacer que la red aprenda a predecir estas etiquetas a partir de las características, logrando una aproximación efectiva con un error cuadrático medio bajo, lo que indica que la red pudo clasificar correctamente las frutas según sus atributos.